## Imports

In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import utils as ut

## Reading

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)
with open(Path('conf') / 'com_comprar_kw.yaml') as file:
    com_comprar = yaml.full_load(file)
with open(Path('conf') / 'product_list.yaml') as file:
    product_cat = yaml.full_load(file)
with open(Path('conf') / 'payment_list.yaml') as file:
    payment_cat = yaml.full_load(file)
    
data         = pd.read_csv(Path(config['input_path']) / "abastiment.csv", sep=",").fillna('')
locations_df = pd.read_csv(Path(config['input_path']) /  'municipis_merge.csv').fillna('')

print(data.shape)

data.head()

(226, 8)


,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat


In [3]:
stopwords = ut.get_all_stopwords()

In [4]:
delivery_patt = ['repart', 'domicil', 'envi', 'recoll', 'dist']

In [5]:

imp_cols = ['COM COMPRAR', 'OBSERVACIONS','PRODUCTE(S)']

In [6]:
com_typos = {
    'Al Urgell':'Alt Urgell',
    'Bages-Moianès':'Moianès',
    'Moianes-Bages':'Moianès',
    'Barcelona':'Barcelonès',
    'Maresme-Barcelonès':'Maresme',
    'Tarragona':'Tarragonès',
    'Baix Montseny':'Vallès Oriental',
    'Baixa Cerdanya':'Cerdanya',
    'Vall Aran':"Vall d'Aran",
    'Alt Maresme':'Maresme',
    'Penedès':'Alt Penedès',
    'Lluçanès':'Osona', #should we consider it a comarca?
            }

## Pre-process columns

In [7]:
data['comarca_origin'] = data['COMARCA'].replace(com_typos)

In [8]:
data['comarca_origin'] = data['comarca_origin'].apply(lambda x: ut.check_comarca_spelling(
    x,locations_df['Comarca'],stopwords) if x not in locations_df['Comarca'] else x)

Not found: Repartim al Bages, Solsonès, Barcelonès i Berguedà


In [9]:
ut.run_preprocess_on_cols(data,imp_cols+['comarca_origin'],stopwords)
ut.run_preprocess_on_cols(locations_df,['Municipi', 'Comarca', 'Capital', 'Provincia'],stopwords)

## New columns

### Payment

In [10]:
data = ut.get_payment_methods(data,imp_cols)

### Locations

In [11]:
# Dictionary to translate municipis to comarca
mun_to_com_dict = locations_df[locations_df['Municipi']!=''].set_index('Municipi')['Comarca'].to_dict()

In [12]:
ut.run_text_locations(data, locations_df, imp_cols, delivery_patt)

In [13]:
data = ut.create_donde_col(data,mun_to_com_dict)

### Binary variables

In [14]:
# create binary variables representing whether they have a payment method or not
ut.create_binary_var(data,payment_cat,'PAGO')
# create binary variables representing whether they have a type of product (1) or not
ut.create_binary_var(data,product_cat,'PRODUCTE(S)'+'_prep')
# create binary variables representing whether they have a type payment method, contact info... (1) or not
ut.create_binary_var(data,com_comprar,'COM COMPRAR'+'_prep')

# improve the website and social network searches adding another column
data.loc[(data['web']!=1) & (data['OBSERVACIONS'+'_prep'].str.contains(
    r'\b'+r'\b|\b'.join(com_comprar['web'])+r'\b')),'web'] = 1
data.loc[(data['socialnet']!=1) & (data['OBSERVACIONS'+'_prep'].str.contains(
    r'\b'+r'\b|\b'.join(com_comprar['socialnet'])+r'\b')),'socialnet'] = 1

### Numerical columns

In [15]:
data = ut.add_numerical_cols(data)

## Save

In [17]:
data.drop(['PRODUCTE(S)_prep','OBSERVACIONS_prep','COM COMPRAR_prep'],axis=1
         ).to_csv(Path(config['input_path']) / 'abastiment_clean.csv', index=False)

In [18]:
print(data.columns)
data.head()

Index(['PROJECTE', 'URGENT', 'PRODUCTE(S)', 'MUNICIPI', 'COM COMPRAR',
       'OBSERVACIONS', 'CCPAE', 'COMARCA', 'comarca_origin',
       'COM COMPRAR_prep', 'OBSERVACIONS_prep', 'PRODUCTE(S)_prep', 'PAGO',
       'DONDE', 'paym_card', 'paym_cash', 'paym_bizum', 'paym_transf', 'fruit',
       'vegetables', 'legumes', 'meat', 'charcuterie', 'mushrooms', 'rice',
       'flour_cereals', 'oil_olives_vinager', 'eggs', 'dairies',
       'herbs_spices', 'hygiene_medicines', 'alcohol',
       'fruit_veggies_products', 'drinks', 'flowers', 'bread_pastries',
       'pasta', 'others', 'iseco', 'delivery', 'mail', 'web', 'pickup', 'shop',
       'market', 'phone', 'orders', 'whatsapp', 'socialnet', 'n_main_prod',
       'n_other_prod', 'n_tot_prod', 'n_paym_methods', 'n_comarcas_delivery'],
      dtype='object')


,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA,comarca_origin,COM COMPRAR_prep,...,market,phone,orders,whatsapp,socialnet,n_main_prod,n_other_prod,n_tot_prod,n_paym_methods,n_comarcas_delivery
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona,Osona,repartim domicili osona resta catalunya consul...,...,0,0,0,0,0,0,3,3,0,2
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya,Cerdanya,contacta telefon mail xarxes socials cabres mi...,...,0,1,0,0,1,0,1,1,0,1
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre,Ribera d'Ebre,agrobotiga dijous divendres dijous tarde repar...,...,0,0,0,0,0,1,0,1,0,1
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona,Barcelonès,venda directa punts recollida tota catalunya d...,...,0,0,0,0,0,0,1,1,0,1
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat,Baix Llobregat,traves whatsap telefon xxxxxxxxx agrobotiga c ...,...,0,1,0,1,0,2,2,4,0,1
